In [1]:
import numpy as np
import pandas as pd

from asteroid_dataframe import load_ast_data, calc_ast_data, spline_ast_vec_df, spline_ast_dir
from asteroid_data import orbital_element_batch
from ztf_data import load_ztf_det_all, ztf_reindex, load_ztf_nearest_ast, make_ztf_near_elt, calc_hit_freq
from astro_utils import deg2dist

from typing import Dict

In [2]:
n0=1
n1=65

mjd0 = 58270.0
mjd1 = 58906.0

progbar=True

In [3]:
# Load data in this date range
df_ast_load, df_earth_load, df_sun_load = load_ast_data(n0=n0, n1=n1, mjd0=mjd0, mjd1=mjd1, progbar=progbar)

Loading asteroid data from n0=1 to n1=65 in 1 blocks...


In [4]:
# df_ast_load

In [5]:
# Orbital elements for first 64 asteroids (dict of numpy arrays)
elts = orbital_element_batch(np.arange(1, 65))

In [6]:
# Calculate vectors from these elements on the fly with calc_ast_data
df_ast_calc, df_earth_calc, df_sun_calc = calc_ast_data(elts=elts, mjd0=mjd0, mjd1=mjd1, element_id=None)

In [7]:
# df_ast_calc

## Splined Direction at ZTF Dates

In [8]:
ztf, mjd_unq = load_ztf_det_all()

In [9]:
ztf

,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,uz,mag_app,asteroid_prob
0,b'ZTF18acebhfp',676397301515010013,14490,58430.397303,41.357345,58.879488,0.387942,0.653853,0.649598,18.946699,0.865682
1,b'ZTF18abodmwk',596403415715010014,5831,58350.403414,30.969721,65.305308,0.358224,0.558644,0.748059,19.010401,0.855504
2,b'ZTF18abodmwk',626428345715010011,10614,58380.428345,30.969705,65.305294,0.358224,0.558644,0.748059,18.935900,0.855504
3,b'ZTF18abodmwk',630507595715015045,11250,58384.507593,30.969940,65.305305,0.358223,0.558645,0.748059,19.260401,0.855504
4,b'ZTF18abodmwk',618384965715010022,9040,58372.384965,30.969643,65.305179,0.358226,0.558644,0.748058,19.220200,0.855504
...,...,...,...,...,...,...,...,...,...,...,...
5697957,b'ZTF20aareruw',1151532523515015015,97109,58905.532523,253.007910,55.485537,-0.165587,-0.169403,0.971537,19.192400,0.608023
5697958,b'ZTF20aarerwx',1151533002615015009,97110,58905.533009,232.886408,53.509617,-0.358833,-0.115301,0.926253,19.687099,0.559474
5697959,b'ZTF20aarerww',1151533002115010003,97110,58905.533009,236.167899,54.618457,-0.322375,-0.116973,0.939357,19.957001,0.392662
5697960,b'ZTF20aarervr',1151526063515015015,97098,58905.526065,286.235286,33.876902,0.232120,-0.509626,0.828494,19.049299,0.517241


In [10]:
# mjd for the spline output
mjd = mjd_unq

In [11]:
df_ast_load_out, df_earth_load_out, df_sun_load_out = spline_ast_vec_df(df_ast=df_ast_load, df_earth=df_earth_load, df_sun=df_sun_load, mjd=mjd, include_elts=False)

In [12]:
# df_ast_load_out

In [13]:
df_ast_calc_out, df_earth_calc_out, df_sun_calc_out = spline_ast_vec_df(df_ast=df_ast_calc, df_earth=df_earth_calc, df_sun=df_sun_calc, mjd=mjd, include_elts=False)

In [14]:
# df_ast_calc_out

## Assemble Batch of Asteroids Near Orbital Elements

In [15]:
ztf_ast = load_ztf_nearest_ast()

In [16]:
# Asteroid numbers and hit counts
ast_num, hit_count = calc_hit_freq(ztf=ztf_ast, thresh_sec=2.0)

# Sort the hit counts in descending order and find the top 64
idx = np.argsort(hit_count)[::-1][0:64]

# Extract the asteroid number and hit count for this batch
ast_nums_best = ast_num[idx]
hit_count_best = hit_count[idx]

In [17]:
# Orbital elements for best asteroids (dict of numpy arrays)
element_id_best = np.sort(ast_nums_best)
elts_best = orbital_element_batch(element_id_best)

# Extract the epoch
# epoch = elts['epoch'][0]

In [18]:
# Compute mjd0 and mjd1 from mjd_unq
mjd0 = np.floor(np.min(mjd_unq))
mjd1 = np.ceil(np.max(mjd_unq))
# other arguments
progbar=True

# Calculate positions in this date range, sapmled daily
df_ast_daily, df_earth_daily, df_sun_daily = calc_ast_data(elts=elts_best, mjd0=mjd0, mjd1=mjd1, element_id=element_id_best)

In [19]:
# df_ast_daily.iloc[561:562]

In [20]:
# Spline positions at ztf times
mjd = mjd_unq
df_ast, df_earth, df_sun = spline_ast_vec_df(df_ast=df_ast_daily, df_earth=df_earth_daily, df_sun=df_sun_daily, mjd=mjd, include_elts=False)

In [21]:
# add time stamp for debugging
time_stamp_id = np.tile(np.arange(97111, dtype=np.int32), 64)
df_ast['TimeStampID'] = time_stamp_id

# mask for debug row
mask = (df_ast.element_id==733) & (df_ast.TimeStampID == 76978)

In [22]:
# df_ast[mask]

In [23]:
mask_daily = (df_ast_daily.element_id == 733) & (df_ast_daily.mjd == 58831)
# df_ast_daily[mask_daily]

In [24]:
df_dir = spline_ast_dir(df_ast=df_ast, df_earth=df_earth, site_name='palomar')

In [25]:
# add time stamp id for debugging
df_dir['TimeStampID'] = time_stamp_id

In [26]:
mask = (df_dir.element_id==733) & (df_dir.TimeStampID == 76978)
df_dir[mask]

,element_id,mjd,ra,dec,ux,uy,uz,delta,TimeStampID
76978,733,58831.098241,33.954411,41.048796,0.625555,0.647681,0.434961,2.683573,76978


In [27]:
# ztf

In [28]:
thresh_deg = 1.0
ztf_tbl = make_ztf_near_elt(ztf=ztf, df_dir=df_dir, thresh_deg=thresh_deg)

In [29]:
ztf_tbl.keys()

dict_keys([733, 1476, 1803, 2015, 2294, 3426, 3904, 4133, 5179, 5817, 6271, 6455, 9229, 9262, 9936, 10032, 10368, 10555, 11952, 13706, 14348, 14692, 15786, 17262, 18181, 19794, 25318, 26793, 27860, 30935, 31539, 31841, 43796, 48453, 49627, 51921, 53516, 59123, 59244, 59245, 62002, 72911, 73961, 76841, 77774, 85295, 85937, 90777, 92136, 103501, 104194, 112578, 114238, 120355, 122263, 134815, 142999, 161094, 178441, 203722, 253246, 306781, 313521, 324582])

In [30]:
ztf_i = ztf_tbl[733]
ztf_i

,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,uz
37606,b'ZTF18abtdsvv',621336711115010010,9577,58375.336713,350.405941,0.879484,0.985897,-0.146790,0.080371
37607,b'ZTF18abtdsvv',621309991115010019,9572,58375.310000,350.405990,0.879483,0.985897,-0.146789,0.080371
37610,b'ZTF18abtdsvv',614358404915010002,8240,58368.358403,350.405972,0.879528,0.985897,-0.146789,0.080371
37612,b'ZTF18abtdsvv',615324371115015021,8425,58369.324375,350.405987,0.879540,0.985897,-0.146789,0.080371
141833,b'ZTF18abwpfho',623339405515010000,9980,58377.339410,348.764546,0.189321,0.980829,-0.177448,0.080534
...,...,...,...,...,...,...,...,...,...
5461148,b'ZTF18adccvqy',1145163955415010017,95282,58899.163958,40.031822,35.961235,0.619758,0.711255,0.331687
5461229,b'ZTF20aapeoax',1145163955215015011,95282,58899.163958,41.260103,36.441613,0.604733,0.723029,0.333956
5646481,b'ZTF19abjairu',1142179350215010035,94593,58896.179352,40.098806,37.053481,0.610474,0.711312,0.348363
5646482,b'ZTF19abjairu',1145163030215010041,95280,58899.163032,40.099006,37.053438,0.610473,0.711314,0.348361


In [31]:
thresh_dist = deg2dist(thresh_deg)
mask_test = (ztf_ast.nearest_ast_num == 733) #& (ztf_ast.nearest_ast_dist < thresh_dist)
ztf_test = ztf_ast[mask_test]
ztf_test

,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,uz,mag_app,asteroid_prob,nearest_ast_num,nearest_ast_dist,ast_ra,ast_dec,ast_ux,ast_uy,ast_uz
395815,b'ZTF19abjphnr',939445724715015021,41296,58693.445729,44.285479,33.579335,0.596406,0.753718,0.276059,15.9857,1.0,733,0.000004,44.285291,33.579191,0.596409,0.753717,0.276058
420554,b'ZTF19abjqzrj',939485674715015012,41349,58693.485671,44.293555,33.585205,0.596283,0.753793,0.276120,15.5081,1.0,733,0.000004,44.293377,33.585051,0.596286,0.753791,0.276118
430852,b'ZTF19abkkmjx',934476402915015003,40253,58688.476401,43.232256,32.855154,0.612042,0.743715,0.268871,16.0814,1.0,733,0.000004,43.232065,32.855006,0.612045,0.743713,0.268869
433118,b'ZTF19abkkmjx',934476861615015007,40254,58688.476863,43.232370,32.855227,0.612041,0.743716,0.268872,16.0991,1.0,733,0.000004,43.232164,32.855074,0.612044,0.743714,0.268870
549984,b'ZTF19ablpetd',942451704715015001,42140,58696.451701,44.889994,34.015204,0.587237,0.759236,0.280557,15.7284,1.0,733,0.000004,44.889810,34.015059,0.587240,0.759235,0.280555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5242925,b'ZTF20aanbfry',1139159645615015005,93432,58893.159641,38.972055,36.841868,0.622190,0.700314,0.349914,15.5513,1.0,733,0.000003,38.971815,36.841881,0.622192,0.700311,0.349915
5364815,b'ZTF20aaoeplv',1142123785315015008,94493,58896.123785,39.695137,36.790572,0.616202,0.707510,0.346013,15.3835,1.0,733,0.000003,39.694888,36.790585,0.616204,0.707508,0.346014
5381516,b'ZTF20aaoggbf',1142182525315015008,94597,58896.182523,39.709609,36.789589,0.616080,0.707654,0.345936,15.6018,1.0,733,0.000004,39.709351,36.789603,0.616082,0.707651,0.345937
5446528,b'ZTF20aapdbbp',1145110625315015005,95182,58899.110625,40.456356,36.751514,0.609662,0.715006,0.342168,15.2027,1.0,733,0.000003,40.456109,36.751523,0.609665,0.715003,0.342169


## Test that Near Batch is a Superset of Observations with Matching Nearest Asteroid Number

In [32]:
np.max(ztf_test.nearest_ast_dist)

0.014939042881650651

In [33]:
np.argmax(ztf_test.nearest_ast_dist)

111

In [34]:
# ztf_test.iloc[111]

In [35]:
ztf_test.loc[3726495:3726496]

,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,uz,mag_app,asteroid_prob,nearest_ast_num,nearest_ast_dist,ast_ra,ast_dec,ast_ux,ast_uy,ast_uz
3726495,b'ZTF19acyklxa',1077098243515015057,76978,58831.098241,32.861095,40.824081,0.635636,0.63676,0.436466,20.046301,0.547102,733,0.014939,33.954415,41.048797,0.625554,0.647681,0.434961


In [39]:
index_hit = ztf_test.index

In [38]:
ztf_i.loc[index_hit]

,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,uz
395815,b'ZTF19abjphnr',939445724715015021,41296,58693.445729,44.285479,33.579335,0.596406,0.753718,0.276059
420554,b'ZTF19abjqzrj',939485674715015012,41349,58693.485671,44.293555,33.585205,0.596283,0.753793,0.276120
430852,b'ZTF19abkkmjx',934476402915015003,40253,58688.476401,43.232256,32.855154,0.612042,0.743715,0.268871
433118,b'ZTF19abkkmjx',934476861615015007,40254,58688.476863,43.232370,32.855227,0.612041,0.743716,0.268872
549984,b'ZTF19ablpetd',942451704715015001,42140,58696.451701,44.889994,34.015204,0.587237,0.759236,0.280557
...,...,...,...,...,...,...,...,...,...
5242925,b'ZTF20aanbfry',1139159645615015005,93432,58893.159641,38.972055,36.841868,0.622190,0.700314,0.349914
5364815,b'ZTF20aaoeplv',1142123785315015008,94493,58896.123785,39.695137,36.790572,0.616202,0.707510,0.346013
5381516,b'ZTF20aaoggbf',1142182525315015008,94597,58896.182523,39.709609,36.789589,0.616080,0.707654,0.345936
5446528,b'ZTF20aapdbbp',1145110625315015005,95182,58899.110625,40.456356,36.751514,0.609662,0.715006,0.342168
